In [1]:
!pip install fastapi uvicorn pyngrok transformers --quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ngrok config add-authtoken 2ciD2bCe3EcRQFXuZ6FIDBM3Bz8_4JKZEpL4TcXkG8Z1bHcaK

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
!pip install nest_asyncio

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from pyngrok import ngrok

# Apply nest_asyncio to allow running asyncio in the notebook
nest_asyncio.apply()

app = FastAPI()

# Define the path to the model in Google Drive
LOCAL_MODEL_PATH = "/content/drive/MyDrive/Emergency_hackaton/zephyr"  # Update with your actual path

# Load model and tokenizer from Google Drive
tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL_PATH, device_map="auto")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

class Prompt(BaseModel):
    prompt: str

@app.post("/generate")
async def generate(prompt: Prompt):
    output = generator(prompt.prompt, max_length=150, do_sample=True, temperature=0.7)[0]["generated_text"]
    return {"response": output}

# Expose via ngrok (this will create a public URL)
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the FastAPI app using uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]